In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

In [ ]:
train = pd.read_csv("data/processed/train.csv")
test = pd.read_csv("data/processed/test.csv")

In [ ]:
base_RoBERTa = pd.read_csv("output/RoBERTa_base.csv")

In [ ]:
accuracy = accuracy_score(test["sentiment"], test["zero-shot"])
print(f"Accuracy: {accuracy:.4f}")

cm = confusion_matrix(test["sentiment"], test["zero-shot"], labels=["positive", "negative"])
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["positive", "negative"], yticklabels=["positive", "negative"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
def get_metrics(group):
    accuracy = accuracy_score(group["sentiment"], group["zero-shot"])
    f1 = f1_score(group["sentiment"], group["zero-shot"], pos_label="positive")
    return pd.Series({"accuracy": accuracy, "f1_score": f1})

In [ ]:
perf_by_rating = test.groupby('rating').apply(get_metrics, include_groups=False)
print(perf_by_rating)

In [ ]:
test['review_length'] = test['text'].apply(len)
test['review_length'] = pd.qcut(test['review_length'], q=25)
perf_by_length = test.groupby('review_length', observed=True).apply(get_metrics, include_groups=False)
print(perf_by_length)